In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
pd.options.display.max_columns = 100
    

In [3]:
# Load the data into a pandas dataframe
train_data = pd.read_csv("data/fangraphs/batters/batters_19_21.csv")
test_data = pd.read_csv("data/fangraphs/batters/batters_22.csv")

In [4]:
# Runs Model

# Preprocess the data and select features for the model
# ... (fill in missing values, handle outliers, etc.)
train_data_chosen_cols = train_data[['G_x','HR_x', 'R_x',
       'SB_x', 'BB%_x', 'K%_x', 'ISO_x', 'BABIP', 'OBP_x', 'SLG_x',
       'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
       'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS',
       'PA', 'BB/K', 'AVG',
       'OPS', 'Spd', 'UBR',
       'wSB', 'wRC', 'wRAA',
       'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
       'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
       #'Team',
       'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
       'CSW%']]
train_data_chosen_cols = train_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})

test_data_chosen_cols = test_data[['G_x','HR_x', 'R_x',
       'SB_x', 'BB%_x', 'K%_x', 'ISO_x', 'BABIP', 'OBP_x', 'SLG_x',
       'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
       'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS',
       'PA', 'BB/K', 'AVG',
       'OPS', 'Spd', 'UBR',
       'wSB', 'wRC', 'wRAA',
       'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
       'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
       #'Team', 
       'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
       'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
       'CSW%']]
test_data_chosen_cols = test_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})


In [4]:
train_data_chosen_cols.columns

Index(['G', 'HR', 'R', 'SB', 'BB%', 'K%', 'ISO', 'BABIP', 'OBP', 'SLG', 'wOBA',
       'wRC+', 'BsR', 'Off', 'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO',
       'HBP', 'CS', 'PA', 'BB/K', 'AVG', 'OPS', 'Spd', 'UBR', 'wSB', 'wRC',
       'wRAA', 'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH',
       'IFH%', 'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%',
       'Hard%', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%', 'Z-Contact%',
       'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%', 'CSW%'],
      dtype='object')

In [5]:
train_data_chosen_cols.head(5)

,G,HR,R,SB,BB%,K%,ISO,BABIP,OBP,SLG,wOBA,wRC+,BsR,Off,AB,H,1B,2B,3B,BB,IBB,SO,HBP,CS,PA,BB/K,AVG,OPS,Spd,UBR,wSB,wRC,wRAA,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH,IFH%,BUH,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,CStr%,CSW%
0,348,76,260,27,19.5%,16.8%,0.257,0.327,0.440,0.561,0.416,159,-4.3,110.7,1198,364,215,66,7,294,38,253,6,10,1509,1.16,0.304,1.001,4.7,-3.7,-0.2,312,125.6,0.327,1.47,19.9%,47.7%,32.4%,5.5%,24.7%,25,5.5%,1,20.0%,34.9%,39.4%,25.6%,14.3%,45.2%,40.5%,19.7%,65.2%,37.7%,63.4%,87.7%,80.1%,39.6%,53.9%,7.5%,18.1%,25.6%
1,329,59,249,79,7.2%,17.6%,0.213,0.355,0.370,0.530,0.379,135,14.0,81.1,1349,428,271,86,12,106,4,259,11,14,1474,0.41,0.317,0.900,7.1,8.2,8.7,259,76.3,0.355,1.37,20.6%,45.9%,33.5%,11.1%,16.0%,50,9.9%,1,100.0%,38.2%,35.9%,26.0%,16.8%,46.9%,36.2%,30.9%,71.4%,47.8%,63.1%,88.8%,79.1%,41.8%,64.6%,10.0%,16.8%,26.7%
2,273,81,210,52,10.1%,27.6%,0.303,0.347,0.369,0.596,0.399,153,13.7,94.6,1036,303,159,55,8,119,8,324,12,13,1175,0.37,0.292,0.965,6.6,7.9,4.0,226,80.8,0.347,1.22,19.9%,44.1%,36.1%,7.8%,31.4%,37,11.7%,2,40.0%,45.3%,33.6%,21.1%,15.3%,37.1%,47.6%,32.0%,76.0%,49.1%,50.3%,80.4%,68.4%,38.9%,60.5%,15.5%,12.4%,28.0%
3,339,76,224,61,10.8%,14.2%,0.261,0.262,0.350,0.528,0.365,131,14.9,70.5,1253,334,168,81,9,155,13,204,12,11,1432,0.76,0.267,0.877,6.4,5.5,6.5,234,57.4,0.262,0.72,19.7%,33.6%,46.7%,13.0%,15.4%,32,9.0%,2,66.7%,52.9%,28.5%,18.7%,18.4%,42.7%,38.9%,26.8%,65.1%,42.4%,77.4%,92.1%,86.6%,40.8%,53.4%,5.7%,17.9%,23.6%
4,223,70,174,14,17.4%,22.0%,0.336,0.320,0.431,0.631,0.431,174,10.0,102.4,786,232,113,44,5,172,23,217,21,3,987,0.79,0.295,1.062,4.8,4.7,0.8,218,94.6,0.320,0.57,25.6%,26.9%,47.5%,9.1%,25.5%,17,11.0%,0,0.0%,42.1%,34.0%,23.9%,12.3%,45.2%,42.5%,20.0%,61.2%,37.2%,67.7%,86.9%,80.9%,41.7%,55.2%,7.1%,19.7%,26.8%


In [5]:
cols_to_replace = ['BB%', 'K%', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%','Med%','Hard%', 'O-Swing%','Z-Swing%','Swing%','O-Contact%','Z-Contact%', 'Contact%','Zone%','F-Strike%','SwStr%','CStr%','CSW%']
train_data_chosen_cols[cols_to_replace] = train_data_chosen_cols[cols_to_replace].replace({'%':''}, regex=True)
test_data_chosen_cols[cols_to_replace] = test_data_chosen_cols[cols_to_replace].replace({'%':''}, regex=True)

In [7]:
train_data.head(5)

,Unnamed: 0,Name,Team_x,G_x,PA_x,HR_x,R_x,RBI_x,SB_x,BB%_x,K%_x,ISO_x,BABIP_x,AVG_x,OBP_x,SLG_x,wOBA_x,xwOBA,wRC+_x,BsR,Off,Def,WAR,playerid_x,Team_y,G_y,AB,PA_y,H,1B,2B,3B,HR_y,R_y,RBI_y,BB,IBB,SO,HBP,SF,SH,GDP,SB_y,CS,AVG_y,playerid_y,Team_x.1,PA,BB%_y,K%_y,BB/K,AVG,OBP_y,SLG_y,OPS,ISO_y,Spd,BABIP_y,UBR,wGDP,wSB,wRC,wRAA,wOBA_y,wRC+_y,playerid_x.1,Team_y.1,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH,IFH%,BUH,BUH%,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,playerid_y.1,Team,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,CStr%,CSW%,playerid
0,0,Juan Soto,WSN,348,1509,76,260,242,27,19.5%,16.8%,0.257,0.327,0.304,0.440,0.561,0.416,NaN,159,-4.3,110.7,-4.0,15.3,20123,WSN,348,1198,1509,364,215,66,7,76,260,242,294,38,253,6,11,0,35,27,10,0.304,20123,WSN,1509,19.5%,16.8%,1.16,0.304,0.440,0.561,1.001,0.257,4.7,0.327,-3.7,-0.4,-0.2,312,125.6,0.416,159,20123,WSN,0.327,1.47,19.9%,47.7%,32.4%,5.5%,24.7%,25,5.5%,1,20.0%,34.9%,39.4%,25.6%,14.3%,45.2%,40.5%,20123,WSN,19.7%,65.2%,37.7%,63.4%,87.7%,80.1%,39.6%,53.9%,7.5%,18.1%,25.6%,20123
1,1,Trea Turner,- - -,329,1474,59,249,175,79,7.2%,17.6%,0.213,0.355,0.317,0.370,0.530,0.379,NaN,135,14.0,81.1,11.0,13.7,16252,- - -,329,1349,1474,428,271,86,12,59,249,175,106,4,259,11,8,0,33,79,14,0.317,16252,- - -,1474,7.2%,17.6%,0.41,0.317,0.370,0.530,0.900,0.213,7.1,0.355,8.2,-2.9,8.7,259,76.3,0.379,135,16252,- - -,0.355,1.37,20.6%,45.9%,33.5%,11.1%,16.0%,50,9.9%,1,100.0%,38.2%,35.9%,26.0%,16.8%,46.9%,36.2%,16252,- - -,30.9%,71.4%,47.8%,63.1%,88.8%,79.1%,41.8%,64.6%,10.0%,16.8%,26.7%,16252
2,2,Fernando Tatis Jr.,SDP,273,1175,81,210,195,52,10.1%,27.6%,0.303,0.347,0.292,0.369,0.596,0.399,NaN,153,13.7,94.6,3.8,13.6,19709,SDP,273,1036,1175,303,159,55,8,81,210,195,119,8,324,12,8,0,15,52,13,0.292,19709,SDP,1175,10.1%,27.6%,0.37,0.292,0.369,0.596,0.965,0.303,6.6,0.347,7.9,1.8,4.0,226,80.8,0.399,153,19709,SDP,0.347,1.22,19.9%,44.1%,36.1%,7.8%,31.4%,37,11.7%,2,40.0%,45.3%,33.6%,21.1%,15.3%,37.1%,47.6%,19709,SDP,32.0%,76.0%,49.1%,50.3%,80.4%,68.4%,38.9%,60.5%,15.5%,12.4%,28.0%,19709
3,3,Jose Ramirez,CLE,339,1432,76,224,232,61,10.8%,14.2%,0.261,0.262,0.267,0.350,0.528,0.365,NaN,131,14.9,70.5,10.9,13.2,13510,CLE,339,1253,1432,334,168,81,9,76,224,232,155,13,204,12,12,0,23,61,11,0.267,13510,CLE,1432,10.8%,14.2%,0.76,0.267,0.350,0.528,0.877,0.261,6.4,0.262,5.5,2.9,6.5,234,57.4,0.365,131,13510,CLE,0.262,0.72,19.7%,33.6%,46.7%,13.0%,15.4%,32,9.0%,2,66.7%,52.9%,28.5%,18.7%,18.4%,42.7%,38.9%,13510,CLE,26.8%,65.1%,42.4%,77.4%,92.1%,86.6%,40.8%,53.4%,5.7%,17.9%,23.6%,13510
4,4,Mike Trout,LAA,223,987,70,174,168,14,17.4%,22.0%,0.336,0.320,0.295,0.431,0.631,0.431,NaN,174,10.0,102.4,-4.3,13.1,10155,LAA,223,786,987,232,113,44,5,70,174,168,172,23,217,21,8,0,6,14,3,0.295,10155,LAA,987,17.4%,22.0%,0.79,0.295,0.431,0.631,1.062,0.336,4.8,0.320,4.7,4.5,0.8,218,94.6,0.431,174,10155,LAA,0.320,0.57,25.6%,26.9%,47.5%,9.1%,25.5%,17,11.0%,0,0.0%,42.1%,34.0%,23.9%,12.3%,45.2%,42.5%,10155,LAA,20.0%,61.2%,37.2%,67.7%,86.9%,80.9%,41.7%,55.2%,7.1%,19.7%,26.8%,10155


In [6]:
# Split the data into training and testing sets
train_data_x = train_data_chosen_cols.drop('R', axis=1)
train_data_y = train_data_chosen_cols['R']
test_data_x = test_data_chosen_cols.drop('R', axis=1)
test_data_y = test_data_chosen_cols['R']

# Scale x variable datasets
scaler = MinMaxScaler()
train_data_x = scaler.fit_transform(train_data_x)
test_data_x = scaler.fit_transform(test_data_x)

# Train a random forest regressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(train_data_x, train_data_y)

# Make predictions on the test set
predictions = rf.predict(test_data_x)

# Assign the predictions to the players in the test set
test_data_y_compare = pd.DataFrame({'Actual_Runs': test_data_y, 'Predicted_Runs': predictions})

# Add name back to test_data_y
test_data_y_compare["Name"] = test_data["Name"]

# Sort the players by their predicted fantasy value
sorted_data = test_data_y_compare.sort_values("Predicted_Runs", ascending=False)

# Display the top 10 players with the highest predicted fantasy value
print(test_data_y_compare[["Name", "Predicted_Runs", "Actual_Runs"]].head(10))


               Name  Predicted_Runs  Actual_Runs
0       Aaron Judge          246.48          133
1     Manny Machado          198.37          100
2     Nolan Arenado          194.40           73
3  Paul Goldschmidt          233.41          106
4   Freddie Freeman          230.01          117
5  Francisco Lindor          208.77           98
6    Yordan Alvarez          194.69           95
7      Mookie Betts          198.93          117
8       Jose Altuve          202.68          103
9     J.T. Realmuto          157.55           75


In [9]:
test_data_y_compare

,Actual_Runs,Predicted_Runs,Name
0,133,245.14,Aaron Judge
1,100,197.39,Manny Machado
2,73,193.42,Nolan Arenado
3,106,225.01,Paul Goldschmidt
4,117,224.95,Freddie Freeman
...,...,...,...
312,36,68.94,Kole Calhoun
313,43,82.62,Franmil Reyes
314,27,38.14,Eddie Rosario
315,29,70.33,Elias Diaz


In [21]:
def predict_batters(model_obj, position, year_to_project, train_dataset, test_dataset):

    # Separate Datasets by Position
    train_dataset = train_dataset[train_dataset['Position'] == position]
    # Separate Datasets by Position
    test_dataset = test_dataset[test_dataset['Position'] == position]

    # Preprocess the data and select features for the model
    train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
    test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']

    # ... (fill in missing values, handle outliers, etc.)
    train_data_chosen_cols = train_dataset[['G_x',
        #'HR_x', 'R_x', 'SB_x', 'RBI_x',
        'BB%_x', 'K%_x', 'ISO_x',
        'OBP_x', 'SLG_x',
        'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
        #'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS', 
        'PA','BB/K', 'AVG',
        'OPS', 'Spd', 'UBR',
        'wSB', 'wRC', 'wRAA',
        'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
        'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
        #'Team',
        'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
        'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
        'CSW%', 'Sum_of_HR_R_RBI_SB']]
    train_data_chosen_cols = train_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})

    test_data_chosen_cols = test_dataset[['G_x',
        #'HR_x', 'R_x', 'SB_x', 'RBI_x',
        'BB%_x', 'K%_x', 'ISO_x',
        'OBP_x', 'SLG_x',
        'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
        #'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS', 
        'PA', 'BB/K', 'AVG',
        'OPS', 'Spd', 'UBR',
        'wSB', 'wRC', 'wRAA',
        'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
        'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
        #'Team',
        'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
        'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
        'CSW%', 'Sum_of_HR_R_RBI_SB']]
    test_data_chosen_cols = test_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})

    #proj_data_chosen_cols = dataset_to_project[['G_x','HR_x', 'R_x',
    #    'SB_x', 'BB%_x', 'K%_x', 'ISO_x', 'BABIP', 'OBP_x', 'SLG_x',
    #    'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
    #    'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS',
    #    'PA', 'BB/K', 'AVG',
    #    'OPS', 'Spd', 'UBR',
    #    'wSB', 'wRC', 'wRAA',
    #    'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
    #    'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
        #'Team', 
    #    'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
    #    'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
    #    'CSW%']]
    #proj_data_chosen_cols = proj_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})

    #train_data_chosen_cols['Sum_of_HR_R_RBI_SB'] = train_data_chosen_cols['HR'] + train_data_chosen_cols['R'] + train_data_chosen_cols['RBI'] + train_data_chosen_cols['SB']
    #test_data_chosen_cols['Sum_of_HR_R_RBI_SB'] = test_data_chosen_cols['HR'] + test_data_chosen_cols['R'] + test_data_chosen_cols['RBI'] + test_data_chosen_cols['SB']
    #proj_data_chosen_cols['Sum_of_HR_R_RBI_SB'] = proj_data_chosen_cols['HR'] + proj_data_chosen_cols['R'] + proj_data_chosen_cols['RBI'] + proj_data_chosen_cols['SB']

    # Remove % signs
    train_data_chosen_cols = train_data_chosen_cols.replace('%','', regex=True)
    test_data_chosen_cols = test_data_chosen_cols.replace('%','', regex=True)

    # Convert cols to floats
    train_data_chosen_cols.astype(np.float64)
    test_data_chosen_cols.astype(np.float64)
    
    #targets = ['Sum_of_HR_R_RBI_SB', 'OBP']
    targets = ['Sum_of_HR_R_RBI_SB']

    for target in targets:
        # Split the data into training and testing sets
        train_data_y = train_data_chosen_cols[target]
        train_data_x = train_data_chosen_cols.drop(targets, axis=1)
        test_data_y = test_data_chosen_cols[target]
        test_data_x = test_data_chosen_cols.drop(targets, axis=1)
        
        # Scale x variable datasets
        scaler = MinMaxScaler()
        train_data_x_scaled = scaler.fit_transform(train_data_x)
        test_data_x_scaled = scaler.fit_transform(test_data_x)

        # ML Model using using fit and predict QB Fantasy Points
        model = model_obj
        model_name = type(model).__name__
        print('Predict '+ position + ' with a '+ model_name + ' model')
        if (model_name == 'LinearRegression') or (model_name == 'Ridge'):
            model = model.fit(train_data_x_scaled,train_data_y)
        else:
            model = model.fit(train_data_x_scaled,train_data_y.values.ravel())
        y_preds = model.predict(test_data_x_scaled)
        
        mean_sq_err = None
        r2_err = None
        if year_to_project == 2023:
            pass
        else:
            # The mean squared error
            mean_sq_err = mean_squared_error(test_data_y, y_preds)
            #print("Mean squared error: %.2f" % mean_sq_err)
            
            # The mean absolute error
            mean_ab_err = mean_squared_error(test_data_y, y_preds)

            # The coefficient of determination: 1 is perfect prediction
            r2_err = r2_score(test_data_y.values.ravel(), y_preds)
            #print("Coefficient of determination: %.2f" % r2_err)
            # Print Importance Coefficients
            #cdf = pd.DataFrame(model_rf.feature_importances_, x_qb_2012_2020.columns, columns=['RF Coefficients'])
            #print(cdf)


        """
        # Plot outputs
        plt.scatter(x_test.iloc[:,0].values, y_test, color="black")
        plt.plot(x_test.iloc[:,0].values, y_preds_rf, '.', color="blue")

        plt.show()
        """

        # Create new dataframe for projections
        player_point_proj = None
        proj_col_name  = 'Model_Projection_'+target
        actuals_col_name  = 'Actual_'+target
        if (model_name == 'LinearRegression') or (model_name == 'Ridge'):
            #player_point_proj = pd.DataFrame({'Age': test_data_x['Age'], 'SLG': test_data_x['SLG'], proj_col_name: y_preds[:,0], actuals_col_name: test_data_y[target]})
            player_point_proj = pd.DataFrame({'PA': test_data_x['PA'], 'SLG': test_data_x['SLG'], proj_col_name: y_preds, actuals_col_name: test_data_y})
        else:
            player_point_proj = pd.DataFrame({'PA': test_data_x['PA'], 'SLG': test_data_x['SLG'], proj_col_name: y_preds, actuals_col_name: test_data_y})
        # Merge player names and info back in
        #player_point_proj_wnames = player_point_proj.merge(master_df[['Player', 'Age', 'Position', 'Year', join_column, 'Fantasy_PPR']], how='inner', left_on=['Age', join_column, 'Actual_Points'], right_on=['Age', join_column, 'Fantasy_PPR'])
        player_point_proj_wnames = player_point_proj.merge(test_dataset[['Name', 'PA', 'SLG_x', target]], how='inner', left_on=['PA', 'SLG'], right_on=['PA', 'SLG_x'])
        player_point_proj_wnames = player_point_proj_wnames.drop(columns=[target])

        # Calculate Model vs Actual Delta
        player_point_proj_wnames['Model_v_Actual_Delta'] = player_point_proj_wnames[proj_col_name] - player_point_proj_wnames[actuals_col_name]

        # Prep Dataframe for csv output
        player_point_proj_wnames = player_point_proj_wnames.sort_values(by=proj_col_name, ascending=False)
        player_point_proj_wnames = player_point_proj_wnames[['Name', 'PA', proj_col_name, actuals_col_name, 'Model_v_Actual_Delta']]
        # Save dataframes
        """
        years = player_point_proj_wnames['Year'].unique()
        #position = player_point_proj_wnames['Position'][0]
        for year in years:
            df = player_point_proj_wnames[player_point_proj_wnames["Year"] == year]
            filename = 'projections/random_forest/'+str(position)+'/_'+str(year)+'_projections.csv'
            df.to_csv(filename)
        """
        if year_to_project == 2023:
            import time
            timestr = time.strftime("%Y%m%d-%H%M%S")
            df = player_point_proj_wnames
            adp = pd.read_csv('data/FantasyPros_2023_Overall_MLB_ADP_Rankings.csv')
            adp = adp.rename({'Player': 'Name'}, axis=1)
            df_w_adp = df.merge(adp, how='left', on='Name')
            df_w_adp = df_w_adp[['Name',proj_col_name, actuals_col_name, 'AVG']]
            df_w_adp = df_w_adp.rename({'AVG': 'ADP'}, axis=1)
            df_w_adp = df_w_adp.drop_duplicates()
            #filename = 'projections/'+str(position)+'/'+str(model_name)+'2022_projections_'+timestr+'.csv'
            filename = 'projections/batters/'+str(position)+'/'+str(model_name)+'2023_projections_'+timestr+'.csv'
            df_w_adp.to_csv(filename)
        else:
            return model_name, mean_sq_err, mean_ab_err, r2_err
        

def fantasy_points_predictor(models, positions, year_to_project, train_dataset, test_dataset):
    import time
    timestr = time.strftime("%Y%m%d-%H%M%S")

    results = []
    for position in positions:
        position_projs_df = []
        for model in models:
            model_name, mean_sq_err, mean_ab_err, r2_err = predict_batters(model, position, year_to_project, train_dataset, test_dataset)
            result = [model_name, position, mean_sq_err, mean_ab_err, r2_err]
            results.append(result)
            #position_projs_df.append(result)
        #avg_position_projs = position_projs_df.groupby('Player')

    results_df = pd.DataFrame(results, columns=['Model Name', 'Position', 'Mean Square Error', 'Mean Absolute Error', 'R2 Score'])
    results_df = results_df.sort_values(by=['Position', 'Mean Absolute Error'], ascending=[True, False])
    c_models = results_df[results_df['Position'] == 'C'] 
    fb_models = results_df[results_df['Position'] == '1B'] 
    sb_models = results_df[results_df['Position'] == '2B'] 
    tb_models = results_df[results_df['Position'] == '3B']
    ss_models = results_df[results_df['Position'] == 'SS']
    of_models = results_df[results_df['Position'] == 'OF']
    dh_models = results_df[results_df['Position'] == 'DH'] 
    top_c_model = c_models.tail(1)
    top_fb_model = fb_models.tail(1)
    top_sb_model = sb_models.tail(1)
    top_tb_model = tb_models.tail(1)
    top_ss_model = ss_models.tail(1)
    top_of_model = of_models.tail(1)
    top_dh_model = dh_models.tail(1)
    results_df_top_performers = top_c_model.append(top_fb_model)
    results_df_top_performers = results_df_top_performers.append(top_sb_model) 
    results_df_top_performers = results_df_top_performers.append(top_tb_model)
    results_df_top_performers = results_df_top_performers.append(top_ss_model) 
    results_df_top_performers = results_df_top_performers.append(top_of_model) 
    results_df_top_performers = results_df_top_performers.append(top_dh_model) 
    
    if year_to_project == 2023:
        print('Simulation complete! Check the projections folder to find your ranked players by position for this years draft.')
    else:
        results_filename = 'projections/batters/2023_model_results_summary_'+timestr+'.csv'
        results_df.to_csv(results_filename)
        top_model_filename = 'projections/batters/2023_top_models_by_position_summary_'+timestr+'.csv'
        results_df_top_performers.to_csv(top_model_filename)
        print('Simulation complete! Check the predictor_tool_results folder to find summary of models.')
        return results_df_top_performers

def model_object_generator(model_name):
    if model_name == 'LinearRegression':
        return LinearRegression()
    elif model_name == 'Ridge':
        return Ridge()
    elif model_name == 'Lasso':
        return Lasso()
    elif model_name == 'BayesianRidge':
        return BayesianRidge()
    elif model_name == 'RandomForestRegressor':
        #return RandomForestRegressor(n_estimators=1000, min_samples_leaf=4, min_samples_split=10)
        return RandomForestRegressor(n_estimators=1000)
    elif model_name == 'KNeighborsRegressor':
        return KNeighborsRegressor()
    elif model_name == 'MLPRegressor':
        return MLPRegressor()
    elif model_name == 'Elastic_Net':
        return ElasticNet()
    else:
        print('Model not supported by model_object_generator function at the moment.')
        return None


In [22]:
models_list = [LinearRegression(), Ridge(), Lasso(), BayesianRidge(), ElasticNet(), RandomForestRegressor(), KNeighborsRegressor(), MLPRegressor(max_iter=1000)]
positions_list = ['C', '1B', '2B', '3B', 'SS', 'OF', 'DH']
train_dataset_2022 = pd.read_csv('data/fangraphs/batters/batters_19_21.csv')
test_dataset_2022 = pd.read_csv('data/fangraphs/batters/batters_22.csv')


top_models = fantasy_points_predictor(models_list, positions_list, 2022, train_dataset_2022, test_dataset_2022)


### Need to fix this part since there is no test dataset for the 2023 projection data
#for index, row in top_models.iterrows():
#    models_list = [model_object_generator(row['Model Name'])]
#    positions_list = [row['Position']]
#    fantasy_points_predictor(models_list, positions_list, 2023)

C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

Predict C with a LinearRegression model
Predict C with a Ridge model
Predict C with a Lasso model
Predict C with a BayesianRidge model
Predict C with a ElasticNet model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

Predict C with a RandomForestRegressor model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

...

c:\Users\mattg\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

Predict DH with a LinearRegression model
Predict DH with a Ridge model
Predict DH with a Lasso model
Predict DH with a BayesianRidge model
Predict DH with a ElasticNet model
Predict DH with a RandomForestRegressor model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

Predict DH with a KNeighborsRegressor model
Predict DH with a MLPRegressor model
Simulation complete! Check the predictor_tool_results folder to find summary of models.


c:\Users\mattg\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:203: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df_top_performers = top_c_model.append(top_fb_model)
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:204: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df_top_performers = results_df_top_performers.append(top_sb_model)
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:205: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df_top_perform

In [23]:
top_models

,Model Name,Position,Mean Square Error,Mean Absolute Error,R2 Score
6,KNeighborsRegressor,C,6501.351220,6501.351220,-2.788929
12,ElasticNet,1B,29025.903928,29025.903928,-7.605460
20,ElasticNet,2B,16731.257932,16731.257932,-8.146407
28,ElasticNet,3B,26052.795875,26052.795875,-7.839864
36,ElasticNet,SS,28536.889860,28536.889860,-8.970565
40,LinearRegression,OF,11780.257691,11780.257691,-3.958700
50,Lasso,DH,27369.010343,27369.010343,-11.865246


In [ ]:
# While still using 'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', and 'CS', the top model objects were:
## C -> ElasticNet()
## 1B -> MLPRegressor()
## 2B -> ElasticNet()
## 3B -> ElasticNet()
## SS -> ElasticNet()
## OF -> ElasticNet()
## DH -> Lasso()


# Once I commented out 'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', and 'CS', the top model objects were:
## C -> kNeighborsRegressor()
## 1B -> ElasticNet()
## 2B -> ElasticNet()
## 3B -> ElasticNet()
## SS -> ElasticNet()
## OF -> ElasticNet()
## DH -> Lasso()

In [24]:
train_dataset = pd.read_csv('data/fangraphs/batters/batters_19_21.csv')
test_dataset = pd.read_csv('data/fangraphs/batters/batters_20_22.csv')

#fantasy_points_predictor([ElasticNet()], ['C'], 2023, train_dataset, test_dataset)
fantasy_points_predictor([KNeighborsRegressor()], ['C'], 2023, train_dataset, test_dataset)

Predict C with a KNeighborsRegressor model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']


TypeError: cannot unpack non-iterable NoneType object

In [25]:
#fantasy_points_predictor([MLPRegressor()], ['1B'], 2023, train_dataset, test_dataset)
fantasy_points_predictor([ElasticNet()], ['1B'], 2023, train_dataset, test_dataset)

Predict 1B with a ElasticNet model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']


TypeError: cannot unpack non-iterable NoneType object

In [26]:
fantasy_points_predictor([ElasticNet()], ['2B'], 2023, train_dataset, test_dataset)

Predict 2B with a ElasticNet model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']


TypeError: cannot unpack non-iterable NoneType object

In [27]:
fantasy_points_predictor([ElasticNet()], ['3B'], 2023, train_dataset, test_dataset)


Predict 3B with a ElasticNet model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']


TypeError: cannot unpack non-iterable NoneType object

In [28]:
fantasy_points_predictor([ElasticNet()], ['SS'], 2023, train_dataset, test_dataset)


Predict SS with a ElasticNet model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']


TypeError: cannot unpack non-iterable NoneType object

In [29]:
fantasy_points_predictor([ElasticNet()], ['OF'], 2023, train_dataset, test_dataset)


Predict OF with a ElasticNet model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']


TypeError: cannot unpack non-iterable NoneType object

In [30]:
fantasy_points_predictor([Lasso()], ['DH'], 2023, train_dataset, test_dataset)

Predict DH with a Lasso model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\3371039363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
c:\Users\mattg\anaconda3\lib

TypeError: cannot unpack non-iterable NoneType object

In [31]:
overall_batter_rankings = pd.read_csv('projections/batters/2023_overall_batter_rankings.csv')

import plotly.express as px

def plot_pos_rank(df, stat, title, groupby_col, plot_filename):
    df_plot = px.scatter(df, x="Proj_Position_Rank", y=stat, color=groupby_col, hover_data=['Name', stat], title=title)
    df_plot.write_html(plot_filename)
    df_plot.show()

plot_pos_rank(overall_batter_rankings, 'Model_Projection_Sum_of_HR_R_RBI_SB', 'Projected Batter Performance vs Position Rank', 'Position', 'plots/2023_overall_batter_projections.html')

In [11]:
def predict_batters_espn(model_obj, position, year_to_project, train_dataset, test_dataset):

    # Separate Datasets by Position
    train_dataset = train_dataset[train_dataset['Position'] == position]
    # Separate Datasets by Position
    test_dataset = test_dataset[test_dataset['Position'] == position]

    # Preprocess the data and select features for the model
    train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
    test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']

    train_dataset['Fantasy_Points'] = train_dataset['HR_x']*4 + train_dataset['3B']*3 + train_dataset['2B']*2 + train_dataset['H'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x'] + train_dataset['BB'] - train_dataset['SO']
    test_dataset['Fantasy_Points'] = test_dataset['HR_x']*4 + test_dataset['3B']*3 + test_dataset['2B']*2 + test_dataset['H'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x'] + test_dataset['BB'] - test_dataset['SO']

    # ... (fill in missing values, handle outliers, etc.)
    train_data_chosen_cols = train_dataset[['G_x',
        #'HR_x', 'R_x', 'SB_x', 'RBI_x',
        'BB%_x', 'K%_x', 'ISO_x',
        'OBP_x', 'SLG_x',
        'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
        #'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS', 
        'PA','BB/K', 'AVG',
        'OPS', 'Spd', 'UBR',
        'wSB', 'wRC', 'wRAA',
        'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
        'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
        #'Team',
        'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
        'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
        'CSW%', 'Sum_of_HR_R_RBI_SB', 'Fantasy_Points']]
    train_data_chosen_cols = train_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})

    test_data_chosen_cols = test_dataset[['G_x',
        #'HR_x', 'R_x', 'SB_x', 'RBI_x',
        'BB%_x', 'K%_x', 'ISO_x',
        'OBP_x', 'SLG_x',
        'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
        #'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS', 
        'PA', 'BB/K', 'AVG',
        'OPS', 'Spd', 'UBR',
        'wSB', 'wRC', 'wRAA',
        'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
        'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
        #'Team',
        'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
        'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
        'CSW%', 'Sum_of_HR_R_RBI_SB', 'Fantasy_Points']]
    test_data_chosen_cols = test_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})

    #proj_data_chosen_cols = dataset_to_project[['G_x','HR_x', 'R_x',
    #    'SB_x', 'BB%_x', 'K%_x', 'ISO_x', 'BABIP', 'OBP_x', 'SLG_x',
    #    'wOBA_x', 'wRC+_x', 'BsR', 'Off', 
    #    'AB', 'H', '1B', '2B', '3B', 'BB', 'IBB', 'SO', 'HBP', 'CS',
    #    'PA', 'BB/K', 'AVG',
    #    'OPS', 'Spd', 'UBR',
    #    'wSB', 'wRC', 'wRAA',
    #    'BABIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH', 'IFH%',
    #    'BUH', 'BUH%', 'Pull%', 'Cent%', 'Oppo%', 'Soft%', 'Med%', 'Hard%',
        #'Team', 
    #    'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%',
    #    'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'CStr%',
    #    'CSW%']]
    #proj_data_chosen_cols = proj_data_chosen_cols.rename(columns={'G_x':'G', 'HR_x':'HR', 'R_x':'R', 'SB_x':'SB', 'BB_x':'BB', 'SB_x':'SB', 'BB%_x':'BB%', 'K%_x':'K%', 'ISO_x':'ISO', 'OBP_x':'OBP', 'SLG_x':'SLG', 'wOBA_x': 'wOBA', 'wRC+_x':'wRC+'})

    #train_data_chosen_cols['Sum_of_HR_R_RBI_SB'] = train_data_chosen_cols['HR'] + train_data_chosen_cols['R'] + train_data_chosen_cols['RBI'] + train_data_chosen_cols['SB']
    #test_data_chosen_cols['Sum_of_HR_R_RBI_SB'] = test_data_chosen_cols['HR'] + test_data_chosen_cols['R'] + test_data_chosen_cols['RBI'] + test_data_chosen_cols['SB']
    #proj_data_chosen_cols['Sum_of_HR_R_RBI_SB'] = proj_data_chosen_cols['HR'] + proj_data_chosen_cols['R'] + proj_data_chosen_cols['RBI'] + proj_data_chosen_cols['SB']

    # Remove % signs
    train_data_chosen_cols = train_data_chosen_cols.replace('%','', regex=True)
    test_data_chosen_cols = test_data_chosen_cols.replace('%','', regex=True)

    # Convert cols to floats
    train_data_chosen_cols.astype(np.float64)
    test_data_chosen_cols.astype(np.float64)
    
    #targets = ['Sum_of_HR_R_RBI_SB', 'OBP']
    targets = ['Fantasy_Points']

    for target in targets:
        # Split the data into training and testing sets
        train_data_y = train_data_chosen_cols[target]
        train_data_x = train_data_chosen_cols.drop(targets, axis=1)
        test_data_y = test_data_chosen_cols[target]
        test_data_x = test_data_chosen_cols.drop(targets, axis=1)
        
        # Scale x variable datasets
        scaler = MinMaxScaler()
        train_data_x_scaled = scaler.fit_transform(train_data_x)
        test_data_x_scaled = scaler.fit_transform(test_data_x)

        # ML Model using using fit and predict QB Fantasy Points
        model = model_obj
        model_name = type(model).__name__
        print('Predict '+ position + ' with a '+ model_name + ' model')
        if (model_name == 'LinearRegression') or (model_name == 'Ridge'):
            model = model.fit(train_data_x_scaled,train_data_y)
        else:
            model = model.fit(train_data_x_scaled,train_data_y.values.ravel())
        y_preds = model.predict(test_data_x_scaled)
        
        mean_sq_err = None
        r2_err = None
        if year_to_project == 2023:
            pass
        else:
            # The mean squared error
            mean_sq_err = mean_squared_error(test_data_y, y_preds)
            #print("Mean squared error: %.2f" % mean_sq_err)
            
            # The mean absolute error
            mean_ab_err = mean_squared_error(test_data_y, y_preds)

            # The coefficient of determination: 1 is perfect prediction
            r2_err = r2_score(test_data_y.values.ravel(), y_preds)
            #print("Coefficient of determination: %.2f" % r2_err)
            # Print Importance Coefficients
            #cdf = pd.DataFrame(model_rf.feature_importances_, x_qb_2012_2020.columns, columns=['RF Coefficients'])
            #print(cdf)


        """
        # Plot outputs
        plt.scatter(x_test.iloc[:,0].values, y_test, color="black")
        plt.plot(x_test.iloc[:,0].values, y_preds_rf, '.', color="blue")

        plt.show()
        """

        # Create new dataframe for projections
        player_point_proj = None
        proj_col_name  = 'Model_Projection_'+target
        actuals_col_name  = 'Actual_'+target
        if (model_name == 'LinearRegression') or (model_name == 'Ridge'):
            #player_point_proj = pd.DataFrame({'Age': test_data_x['Age'], 'SLG': test_data_x['SLG'], proj_col_name: y_preds[:,0], actuals_col_name: test_data_y[target]})
            player_point_proj = pd.DataFrame({'PA': test_data_x['PA'], 'SLG': test_data_x['SLG'], proj_col_name: y_preds, actuals_col_name: test_data_y})
        else:
            player_point_proj = pd.DataFrame({'PA': test_data_x['PA'], 'SLG': test_data_x['SLG'], proj_col_name: y_preds, actuals_col_name: test_data_y})
        # Merge player names and info back in
        #player_point_proj_wnames = player_point_proj.merge(master_df[['Player', 'Age', 'Position', 'Year', join_column, 'Fantasy_PPR']], how='inner', left_on=['Age', join_column, 'Actual_Points'], right_on=['Age', join_column, 'Fantasy_PPR'])
        player_point_proj_wnames = player_point_proj.merge(test_dataset[['Name', 'PA', 'SLG_x', target]], how='inner', left_on=['PA', 'SLG'], right_on=['PA', 'SLG_x'])
        player_point_proj_wnames = player_point_proj_wnames.drop(columns=[target])

        # Calculate Model vs Actual Delta
        player_point_proj_wnames['Model_v_Actual_Delta'] = player_point_proj_wnames[proj_col_name] - player_point_proj_wnames[actuals_col_name]

        # Prep Dataframe for csv output
        player_point_proj_wnames = player_point_proj_wnames.sort_values(by=proj_col_name, ascending=False)
        player_point_proj_wnames = player_point_proj_wnames[['Name', 'PA', proj_col_name, actuals_col_name, 'Model_v_Actual_Delta']]
        # Save dataframes
        """
        years = player_point_proj_wnames['Year'].unique()
        #position = player_point_proj_wnames['Position'][0]
        for year in years:
            df = player_point_proj_wnames[player_point_proj_wnames["Year"] == year]
            filename = 'projections/random_forest/'+str(position)+'/_'+str(year)+'_projections.csv'
            df.to_csv(filename)
        """
        if year_to_project == 2023:
            import time
            timestr = time.strftime("%Y%m%d-%H%M%S")
            df = player_point_proj_wnames
            adp = pd.read_csv('data/FantasyPros_2023_Overall_MLB_ADP_Rankings.csv')
            adp = adp.rename({'Player': 'Name'}, axis=1)
            df_w_adp = df.merge(adp, how='left', on='Name')
            df_w_adp = df_w_adp[['Name',proj_col_name, actuals_col_name, 'AVG']]
            df_w_adp = df_w_adp.rename({'AVG': 'ADP'}, axis=1)
            df_w_adp = df_w_adp.drop_duplicates()
            #filename = 'projections/'+str(position)+'/'+str(model_name)+'2022_projections_'+timestr+'.csv'
            filename = 'espn_projections/batters/'+str(position)+'/'+str(model_name)+'2023_projections_'+timestr+'.csv'
            df_w_adp.to_csv(filename)
        else:
            return model_name, mean_sq_err, mean_ab_err, r2_err
        

def fantasy_points_predictor_espn(models, positions, year_to_project, train_dataset, test_dataset):
    import time
    timestr = time.strftime("%Y%m%d-%H%M%S")

    results = []
    for position in positions:
        position_projs_df = []
        for model in models:
            model_name, mean_sq_err, mean_ab_err, r2_err = predict_batters_espn(model, position, year_to_project, train_dataset, test_dataset)
            result = [model_name, position, mean_sq_err, mean_ab_err, r2_err]
            results.append(result)
            #position_projs_df.append(result)
        #avg_position_projs = position_projs_df.groupby('Player')

    results_df = pd.DataFrame(results, columns=['Model Name', 'Position', 'Mean Square Error', 'Mean Absolute Error', 'R2 Score'])
    results_df = results_df.sort_values(by=['Position', 'Mean Absolute Error'], ascending=[True, False])
    c_models = results_df[results_df['Position'] == 'C'] 
    fb_models = results_df[results_df['Position'] == '1B'] 
    sb_models = results_df[results_df['Position'] == '2B'] 
    tb_models = results_df[results_df['Position'] == '3B']
    ss_models = results_df[results_df['Position'] == 'SS']
    of_models = results_df[results_df['Position'] == 'OF']
    dh_models = results_df[results_df['Position'] == 'DH'] 
    top_c_model = c_models.tail(1)
    top_fb_model = fb_models.tail(1)
    top_sb_model = sb_models.tail(1)
    top_tb_model = tb_models.tail(1)
    top_ss_model = ss_models.tail(1)
    top_of_model = of_models.tail(1)
    top_dh_model = dh_models.tail(1)
    results_df_top_performers = top_c_model.append(top_fb_model)
    results_df_top_performers = results_df_top_performers.append(top_sb_model) 
    results_df_top_performers = results_df_top_performers.append(top_tb_model)
    results_df_top_performers = results_df_top_performers.append(top_ss_model) 
    results_df_top_performers = results_df_top_performers.append(top_of_model) 
    results_df_top_performers = results_df_top_performers.append(top_dh_model) 
    
    if year_to_project == 2023:
        print('Simulation complete! Check the projections folder to find your ranked players by position for this years draft.')
    else:
        results_filename = 'espn_projections/batters/2023_model_results_summary_'+timestr+'.csv'
        results_df.to_csv(results_filename)
        top_model_filename = 'espn_projections/batters/2023_top_models_by_position_summary_'+timestr+'.csv'
        results_df_top_performers.to_csv(top_model_filename)
        print('Simulation complete! Check the predictor_tool_results folder to find summary of models.')
        return results_df_top_performers

def model_object_generator(model_name):
    if model_name == 'LinearRegression':
        return LinearRegression()
    elif model_name == 'Ridge':
        return Ridge()
    elif model_name == 'Lasso':
        return Lasso()
    elif model_name == 'BayesianRidge':
        return BayesianRidge()
    elif model_name == 'RandomForestRegressor':
        #return RandomForestRegressor(n_estimators=1000, min_samples_leaf=4, min_samples_split=10)
        return RandomForestRegressor(n_estimators=1000)
    elif model_name == 'KNeighborsRegressor':
        return KNeighborsRegressor()
    elif model_name == 'MLPRegressor':
        return MLPRegressor()
    elif model_name == 'Elastic_Net':
        return ElasticNet()
    else:
        print('Model not supported by model_object_generator function at the moment.')
        return None


In [12]:
models_list = [LinearRegression(), Ridge(), Lasso(), BayesianRidge(), ElasticNet(), RandomForestRegressor(), KNeighborsRegressor(), MLPRegressor(max_iter=1000)]
positions_list = ['C', '1B', '2B', '3B', 'SS', 'OF', 'DH']
train_dataset_2022 = pd.read_csv('data/fangraphs/batters/batters_19_21.csv')
test_dataset_2022 = pd.read_csv('data/fangraphs/batters/batters_22.csv')


top_models = fantasy_points_predictor_espn(models_list, positions_list, 2022, train_dataset_2022, test_dataset_2022)

C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

Predict C with a LinearRegression model
Predict C with a Ridge model
Predict C with a Lasso model
Predict C with a BayesianRidge model
Predict C with a ElasticNet model
Predict C with a RandomForestRegressor model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

Predict C with a KNeighborsRegressor model
Predict C with a MLPRegressor model


c:\Users\mattg\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

...

c:\Users\mattg\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

Predict DH with a LinearRegression model
Predict DH with a Ridge model
Predict DH with a Lasso model
Predict DH with a BayesianRidge model
Predict DH with a ElasticNet model
Predict DH with a RandomForestRegressor model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

Predict DH with a KNeighborsRegressor model
Predict DH with a MLPRegressor model
Simulation complete! Check the predictor_tool_results folder to find summary of models.


c:\Users\mattg\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:206: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df_top_performers = top_c_model.append(top_fb_model)
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:207: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df_top_performers = results_df_top_performers.append(top_sb_model)
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df_top_perform

In [13]:
train_dataset = pd.read_csv('data/fangraphs/batters/batters_19_21.csv')
test_dataset = pd.read_csv('data/fangraphs/batters/batters_20_22.csv')

#fantasy_points_predictor([ElasticNet()], ['C'], 2023, train_dataset, test_dataset)
fantasy_points_predictor_espn([LinearRegression()], ['C'], 2023, train_dataset, test_dataset)

C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

Predict C with a LinearRegression model


TypeError: cannot unpack non-iterable NoneType object

In [14]:
fantasy_points_predictor_espn([ElasticNet()], ['1B'], 2023, train_dataset, test_dataset)

Predict 1B with a ElasticNet model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

TypeError: cannot unpack non-iterable NoneType object

In [15]:
fantasy_points_predictor_espn([ElasticNet()], ['2B'], 2023, train_dataset, test_dataset)

C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

Predict 2B with a ElasticNet model


TypeError: cannot unpack non-iterable NoneType object

In [16]:
fantasy_points_predictor_espn([ElasticNet()], ['3B'], 2023, train_dataset, test_dataset)

Predict 3B with a ElasticNet model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

TypeError: cannot unpack non-iterable NoneType object

In [19]:
fantasy_points_predictor_espn([ElasticNet()], ['SS'], 2023, train_dataset, test_dataset)

Predict SS with a ElasticNet model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

TypeError: cannot unpack non-iterable NoneType object

In [17]:
fantasy_points_predictor_espn([Lasso()], ['OF'], 2023, train_dataset, test_dataset)

Predict OF with a Lasso model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

TypeError: cannot unpack non-iterable NoneType object

In [18]:
fantasy_points_predictor_espn([Lasso()], ['DH'], 2023, train_dataset, test_dataset)

Predict DH with a Lasso model


C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['Sum_of_HR_R_RBI_SB'] = train_dataset['HR_x'] + train_dataset['R_x'] + train_dataset['RBI_x'] + train_dataset['SB_x']
C:\Users\mattg\AppData\Local\Temp\ipykernel_4868\1870351965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset['Sum_of_HR_R_RBI_SB'] = test_dataset['HR_x'] + test_dataset['R_x'] + test_dataset['RBI_x'] + test_dataset['SB_x']
C:\Users\mattg\AppData\Local

TypeError: cannot unpack non-iterable NoneType object